# AP Research Testing notebook

In [1]:
import qmcpy as qp
import numpy as np


## What am I doing here?

To execute my research plan, I am going to test the performance of several integrals under different point sets in repsponse to different stopping criteria. Below is a list of integrals I am going to implement




Keister Function

Box integral

Linear Function

Baysesian Logistic Regression

Genz Function 

Ishigami Function

Multimodal 2d

Below are the point sets I am going to implement

Digital net (Sobol)

Lattice

Halton

IID

I will use all 4 of them for each integral

## Integrands

### Keister Integral

Below is an implementation of the Keister integral using lattices as a point set, I chose the integral with dimension 5 (5 variables) and a tolerance of 0.0001. The stopping criterion, `CubQMCLatticeG` is a stopping criterion that guarantees integration for lattices.

In [8]:
dim = 5 #dimension
lat = qp.Lattice(dim)
tolerance = 1e-4
gaussian_lattice = qp.Gaussian(lat,mean = 0,covariance=1/2) #true measure
keister = qp.Keister(gaussian_lattice) #transform to integrand
keister_lattice_gauss_g = qp.CubQMCLatticeG(keister,abs_tol=tolerance) #stopping criterion




Below is the integration solution using QMC methods, as a comparison, the actual value of the integral is also listed.

In [9]:
#QMC simulation
solution,data = keister_lattice_gauss_g.integrate()
print(solution)

[1.13597947]


In [12]:
#Actual Integral
print(qp.Keister.exact_integ(qp.Keister,d=5))

1.1353239910124924


### Box Integral

Box integral with dimension 4 and tolerance 0.001. Chosen point set as digital nets. 

In [13]:
d = 4
tol = 1e-3
dnet = qp.DigitalNetB2(dimension=d) #discrete distribution
uniform_dnet = qp.Uniform(dnet) #true measure
box = qp.BoxIntegral(uniform_dnet)
box_dnet_uniform = qp.CubQMCNetG(box,abs_tol = tol)



In [14]:
solution,data = box_dnet_uniform.integrate()

print(solution)

[1.12194272 1.33333333]


### Linear Function

Linear function for dimension 100 using digital nets. 

In [17]:
l = qp.Linear0(qp.DigitalNetB2(100,seed=7))
x = l.discrete_distrib.gen_samples(2**10)
y = l.f(x)
print(y.mean())



-1.1757947504520416e-08


### Bayseian

Bayseian integral for dimension 3 using digital nets

In [18]:
blrcoeffs = qp.BayesianLRCoeffs(qp.DigitalNetB2(3,seed=7),feature_array=np.arange(8).reshape((4,2)),response_vector=[0,0,1,1])
x = blrcoeffs.discrete_distrib.gen_samples(2**10)
y = blrcoeffs.f(x)
print(y.shape)
print(y.mean(0))

(1024, 2, 3)
[[ 0.04639394 -0.01440543 -0.05498496]
 [ 0.02176581  0.02176581  0.02176581]]


### Genz Function

Genz function for dimension 2 with different function types and different coefficients

In [19]:
for kind_func in ['oscilatory','corner-peak']:

    for kind_coeff in [1,2,3]:

        g = qp.Genz(qp.DigitalNetB2(2,seed=7),kind_func=kind_func,kind_coeff=kind_coeff)

        x = g.discrete_distrib.gen_samples(2**14)

        y = g.f(x)

        mu_hat = y.mean()

        print('%-15s %-3d %.3f'%(kind_func,kind_coeff,mu_hat))

oscilatory      1   -0.351
oscilatory      2   -0.380
oscilatory      3   -0.217
corner-peak     1   0.713
corner-peak     2   0.712
corner-peak     3   0.720


### Ishigami

The Ishigami function for dimension 3

In [20]:
ishigami = qp.Ishigami(qp.DigitalNetB2(3,seed=7))
x = ishigami.discrete_distrib.gen_samples(2**10)
y = ishigami.f(x)
print(y.mean())

3.499838620282203


### Multimodal 2D

Multimonodal 2D integrals are also part of the implementation, I have yet to implement this in qmcpy.

In [22]:
#TODO: implement